test

In [9]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
from textblob import TextBlob
from csv_cleaning import load_in, remove_links, remove_nan, clean_data

In [8]:
from csv_cleaning import load_in, remove_links, remove_nan, clean_data
df_raw = load_in()
df_basic = clean_data(df_raw)
print("Length of the dataframe is " + str(len(df_basic)) + "\n")
df_basic.head(3)

Length of the dataframe is 6271



,Song Title,Artist,Lyrics,Year
0,The Battle Of New Orleans,Johnny Horton,In 1814 we took a little trip Along with Colo...,1959.0
1,Mack The Knife,Bobby Darin,"Oh the shark, babe Has such teeth, dear And he...",1959.0
2,Personality,Lloyd Price,Over and over I tried to prove my love to you ...,1959.0


In [6]:
df = df_raw.dropna(subset=['Release Date'])
df = df_raw.dropna(subset=['Lyrics'])

In [7]:
print(len(df))
df.head(1)

6271


,Album,Album URL,Artist,Featured Artists,Lyrics,Media,Rank,Release Date,Song Title,Song URL,Writers,Year,Verbs,Nouns,Adverbs,Corpus,Word Counts,Unique Word Counts
0,Battle of New Orleans,https://genius.com/albums/Johnny-horton/Battle...,Johnny Horton,[],In 1814 we took a little trip Along with Colo...,[{'native_uri': 'spotify:track:0dwpdcQkeZqpuoA...,1,1959-04-01,The Battle Of New Orleans,https://genius.com/Johnny-horton-the-battle-of...,"[{'api_path': '/artists/561913', 'header_image...",1959.0,take take take catch fire keep be be fire begi...,trip bacon bean town gun comin while river of'...,along as ago once more so as ago once more the...,take little trip Colonel Jackson mighty Missis...,435,155


In [8]:
# 
vectorizer = TfidfVectorizer(max_features=10, stop_words='english',token_pattern=r'\b[a-zA-Z]{3,}\b')
tfidf_matrix = vectorizer.fit_transform(df['Lyrics'])
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), 
                        columns=vectorizer.get_feature_names_out())
print(len(tfidf_df))
tfidf_df.head(5)

6271


,baby,don,got,just,know,let,like,love,want,yeah
0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,1.0
1,0.0,0.179567,0.000000,0.900227,0.349594,0.0,0.187430,0.000000,0.000000,0.0
2,0.0,0.000000,0.807469,0.000000,0.000000,0.0,0.000000,0.589910,0.000000,0.0
3,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,0.000000,0.0
4,0.0,0.000000,0.429223,0.493295,0.000000,0.0,0.256764,0.627151,0.336424,0.0


In [14]:
def get_sentiment(lyrics):
    blob = TextBlob(lyrics)
    return blob.sentiment.polarity  # Returns -1 (negative) to 1 (positive)

df['sentiment'] = df['Lyrics'].apply(get_sentiment)
df['subjectivity'] = df['Lyrics'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
df.head(1)

,Album,Album URL,Artist,Featured Artists,Lyrics,Media,Rank,Release Date,Song Title,Song URL,Writers,Year,Verbs,Nouns,Adverbs,Corpus,Word Counts,Unique Word Counts,sentiment,subjectivity
0,Battle of New Orleans,https://genius.com/albums/Johnny-horton/Battle...,Johnny Horton,[],In 1814 we took a little trip Along with Colo...,[{'native_uri': 'spotify:track:0dwpdcQkeZqpuoA...,1,1959-04-01,The Battle Of New Orleans,https://genius.com/Johnny-horton-the-battle-of...,"[{'api_path': '/artists/561913', 'header_image...",1959.0,take take take catch fire keep be be fire begi...,trip bacon bean town gun comin while river of'...,along as ago once more so as ago once more the...,take little trip Colonel Jackson mighty Missis...,435,155,0.066681,0.391962
1,That’s All,https://genius.com/albums/Bobby-darin/That-s-all,Bobby Darin,[],"Oh the shark, babe Has such teeth, dear And he...",[{'native_uri': 'spotify:track:3E5ndyOfO6vFDEI...,2,NaN,Mack The Knife,https://genius.com/Bobby-darin-mack-the-knife-...,"[{'api_path': '/artists/218851', 'header_image...",1959.0,have show have keep know bite billow spread be...,shark babe tooth jackknife babe sight shark to...,pearly just so never never now just just down ...,oh shark babe tooth dear show pearly white jac...,224,145,-0.008757,0.219392
2,“Mr Personality’s” 15 Big Hits,https://genius.com/albums/Lloyd-price/Mr-perso...,Lloyd Price,[],Over and over I tried to prove my love to you ...,"[{'provider': 'youtube', 'start': 0, 'type': '...",3,NaN,Personality,https://genius.com/Lloyd-price-personality-lyrics,"[{'api_path': '/artists/355804', 'header_image...",1959.0,try prove do say get get do cause get get do s...,love friend fool fool personality personality ...,over over over over over over so over now over...,try prove love friend say fool fool cause get ...,215,55,0.433333,0.436111
3,The Greatest Hits Of Frankie Avalon,https://genius.com/albums/Frankie-avalon/The-g...,Frankie Avalon,[],"Hey, Venus! Oh, Venus! Venus, if you will Ple...",[],4,NaN,Venus,https://genius.com/Frankie-avalon-venus-lyrics,"[{'api_path': '/artists/1113175', 'header_imag...",1959.0,send thrill want make take place ask promise g...,girl girl kiss arm girl charm girl sunlight ha...,surely too always as long surely too always as...,hey Venus oh Venus Venus send little girl thri...,166,74,0.374312,0.614620
4,Paul Anka Sings His Big 15,https://genius.com/albums/Paul-anka/Paul-anka-...,Paul Anka,[],I'm just a lonely boy Lonely and blue I'm all ...,[],5,NaN,Lonely Boy,https://genius.com/Paul-anka-lonely-boy-lyrics,[],1959.0,do get think want love love kiss hold like hea...,boy nothing moment love night day boy nothing ...,just all just all just so hard just all,lonely boy lonely blue get think want love yes...,140,68,0.141667,0.537745


In [ ]:
pd.concat(tfidf_df, df[['sentiment', 'subjectivity', 'Year', 'Word Counts', 'Unique Word Counts','Rank']]